# Импорт библиотек

In [1]:
#!pip install keras-tuner

In [2]:
import pandas as pd
import re

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import Sequential


from sklearn.preprocessing import LabelEncoder

from kerastuner.tuners import Hyperband
from kerastuner import HyperParameters
from gensim.models import KeyedVectors

Using TensorFlow backend


C:\Users\Nekon\AppData\Local\Temp\ipykernel_3340\3536822836.py:22: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


# Загрузка данных

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission_data = pd.read_csv('sample_submission.csv')

# Анализ данных

In [4]:
train_data.head()

,Unnamed: 0,Text,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,advice Talk to your neighbours family to excha...,Positive
2,2,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,My food stock is not the only one which is emp...,Positive
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
test_data.head()

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,TRENDING: New Yorkers encounter empty supermar...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,When I couldn't find hand sanitizer at Fred Me...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,Find out how you can protect yourself and love...
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,#Panic buying hits #NewYork City as anxious sh...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,#toiletpaper #dunnypaper #coronavirus #coronav...


In [6]:
submission_data.head()

,id,Sentiment
0,787bc85b-20d4-46d8-84a0-562a2527f684,Neutral
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,Neutral
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,Neutral
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,Neutral
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,Neutral


In [7]:
# Проверка распределения столбца 'Sentiment' в тренировочном датасете
sentiment_distribution = train_data['Sentiment'].value_counts()
sentiment_distribution

Positive              11422
Negative               9917
Neutral                7711
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [8]:
# Проверим наличие пропущенных значений в колонке "Text" обучающего и тестового набора данных
missing_train = train_data["Text"].isnull().sum()
missing_test = test_data["Text"].isnull().sum()

missing_train, missing_test

(1, 0)

In [9]:
# Удаление строк с пропущенными значениями
train_data = train_data.dropna(subset=["Text"])

# Предобработка данных

In [10]:
def preprocess_text(text):
    """
    Предобработка текста: преобразование в нижний регистр, удаление URL, знаков препинания и т.д.
    """
    # Преобразование в нижний регистр
    text = text.lower()
    
    # Удаление URL
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Удаление знаков препинания и чисел
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    return text

# Предобработка текстовых данных
train_data["Processed_Text"] = train_data["Text"].apply(preprocess_text)
test_data["Processed_Text"] = test_data["Text"].apply(preprocess_text)

train_data[["Text", "Processed_Text"]].head()

,Text,Processed_Text
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,menyrbie phil_gahan chrisitv and and
1,advice Talk to your neighbours family to excha...,advice talk to your neighbours family to excha...
2,Coronavirus Australia: Woolworths to give elde...,coronavirus australia woolworths to give elder...
3,My food stock is not the only one which is emp...,my food stock is not the only one which is emp...
4,"Me, ready to go at supermarket during the #COV...",me ready to go at supermarket during the covid...


In [11]:
# Инициализация токенизатора
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_data["Processed_Text"])

# Преобразование текста в последовательности чисел
train_sequences = tokenizer.texts_to_sequences(train_data["Processed_Text"])
test_sequences = tokenizer.texts_to_sequences(test_data["Processed_Text"])

# Определение максимальной длины последовательности
max_length = max([len(seq) for seq in train_sequences])

# Дополнение последовательностей до одной и той же длины
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

train_padded.shape, test_padded.shape

((41158, 62), (3798, 62))

In [12]:
# Инициализация и обучение LabelEncoder
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_data["Sentiment"])

# Проверка уникальных классов
unique_classes = encoder.classes_
unique_classes

array(['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral',
       'Positive', nan], dtype=object)

In [13]:
# Загрузка предварительно обученных векторов слов 
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

# Построение RNN

In [ ]:
# Создание матрицы весов для слоя Embedding
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # используем размерность 300 для word2vec 

for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_vector = word2vec_model[word]
        embedding_matrix[i] = embedding_vector  

# Определение размера словаря и размера вложения
vocab_size = len(tokenizer.word_index) + 1

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix],
                              input_length=max_length, trainable=False),  
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(train_data['Sentiment'].unique()), activation='softmax')
])

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
history = model.fit(train_padded, train_labels, epochs=5, validation_split=0.2, batch_size=32)

In [ ]:
def plot_training_history(history):
    """
    Визуализация истории обучения модели.
    
    Параметры:
    - history: объект истории, возвращенный функцией model.fit().
    """
    plt.figure(figsize=(12, 4))

    # График потерь
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss Evolution')

    # График точности
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy Evolution')

    plt.tight_layout()
    plt.show()
    
plot_training_history(history)

Модель успешно обучается, что видно по уменьшению потерь и увеличению точности на тренировочных данных.

При этом, начиная с пятой эпохи,судя по графикам, модель переобучается, т.к. валидационные потери начинают увеличиваться, а точность немного уменьшается.

Рассмотрим добавление регуляризации и используем метод dropout.

Также рассмотрим возможность обучения модели на большем количестве эпох с ранним остановом на основе валидационных потерь, чтобы предотвратить дальнейшее переобучение.

In [ ]:
# Определение размера словаря и размера вложения
vocab_size = len(tokenizer.word_index) + 189

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix],
                              input_length=max_length, trainable=False),
    tf.keras.layers.Dropout(0.5),  # Dropout слой
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.5),  # Dropout слой
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout слой
    tf.keras.layers.Dense(len(unique_classes), activation='softmax')
])

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callback для раннего останова
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Обучение модели с добавлением callback для раннего останова
history = model.fit(train_padded, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[early_stop])

In [ ]:
plot_training_history(history)

Данная модель также успешно обучается, что видно по уменьшению потерь и увеличению точности на тренировочных данных.

Механизм раннего останова, по всей видимости, успешно прекратил обучение, когда валидационные потери начали расти (после 6-й эпохи), предотвращая дальнейшее переобучение.

Добавление Dropout и раннего останова помогли улучшить производительность модели на валидационных данных.

In [ ]:
# Определение размера словаря и размера вложения
vocab_size = len(tokenizer.word_index) + 1

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix],
                              input_length=max_length, trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),  # Bidirectional LSTM
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),  # Добавляем еще один слой LSTM
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(unique_classes), activation='softmax')
])

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callback для раннего останова
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Обучение модели с добавлением callback для раннего останова
history = model.fit(train_padded, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[early_stop])

In [ ]:
# Предсказание на тестовых данных
test_predictions = model.predict(test_padded)
test_predictions = test_predictions.argmax(axis=1)

# Обратное преобразование предсказанных меток в текстовые значения
predicted_labels = encoder.inverse_transform(test_predictions)

# Создание DataFrame для сохранения предсказаний
submission_data = pd.DataFrame({
    'id': test_data['id'],  # использование столбца 'id' из тестовых данных
    'Sentiment': predicted_labels
})

# Сохранение предсказаний в CSV файл
submission_data.to_csv('predictions.csv', index=False)

In [ ]:
submission_data

### Архитектура RNN, близкая к state of the art для данной задачи

In [ ]:
# Определение размера словаря и размера вложения
vocab_size = len(tokenizer.word_index) + 1


# Создание модели
model = Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix],
                              input_length=max_length, trainable=False),
    Bidirectional(LSTM(64, return_sequences=True)),  # Bidirectional LSTM
    Dropout(0.5),  # Dropout для регуляризации
    Bidirectional(LSTM(32, return_sequences=True)),  # Дополнительный Bidirectional LSTM слой
    Dropout(0.5),  # Dropout для регуляризации
    Bidirectional(LSTM(32)),  # Дополнительный Bidirectional LSTM слой
    Dense(64, activation='relu'),
    Dropout(0.5),  # Dropout для регуляризации
    Dense(len(unique_classes), activation='softmax')
])

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callback для раннего останова
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Обучение модели с добавлением callback для раннего останова
history = model.fit(train_padded, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[early_stop])

### Подбор гипперпараметров

In [ ]:
def build_model(hp: HyperParameters):
    model = tf.keras.models.Sequential()
    
    # Зафиксируем размер embedding слоя, используя веса из Word2Vec
    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=300,  # фиксированный размер вектора из Word2Vec
        weights=[embedding_matrix],  # используем предварительно обученные веса
        input_length=max_length,
        trainable=False  # не обучаем этот слой во время тренировки
    ))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=hp.Int('lstm_units', min_value=32, max_value=128, step=16), 
        return_sequences=True)))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=hp.Int('lstm_units', min_value=32, max_value=128, step=16))))
    model.add(tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=128, step=16), 
        activation='relu'))
    model.add(tf.keras.layers.Dense(len(unique_classes), activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), 
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=5,
    directory='hyperband',
    project_name='text_classification'
)

tuner.search(train_padded, train_labels, epochs=5, validation_split=0.2)

In [ ]:
# Получение информации о лучшей модели
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Вывод лучших гиперпараметров
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('dense_units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

# Получение и компиляция лучшей модели
best_model = tuner.hypermodel.build(best_hps)

# Обучение лучшей модели
best_model.fit(train_padded, train_labels, epochs=best_hps.get('epochs'), validation_split=0.2, batch_size=best_hps.get('batch_size'))

In [ ]:
# Предсказание на тестовых данных
test_predictions_3 = best_model.predict(test_padded)
test_predictions_3 = test_predictions_3.argmax(axis=1)

# Обратное преобразование предсказанных меток в текстовые значения
predicted_labels_3 = encoder.inverse_transform(test_predictions_3)

# Создание DataFrame для сохранения предсказаний
submission_data_3 = pd.DataFrame({
    'id': test_data['id'],  # использование столбца 'id' из тестовых данных
    'Sentiment': predicted_labels_3
})

# Сохранение предсказаний в CSV файл
submission_data_3.to_csv('predictions_3.csv', index=False)